In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/MyDrive/beta_rank

/content/drive/MyDrive/beta_rank


In [3]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.2/517.2 KB 10.5 MB/s eta 0:00:00


In [4]:
!pip install thop

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Rank generation

c10

In [5]:
!python rank_generation.py \
--pretrain_dir './baseline/Baseline_VGG_cifar10/vgg_16_bn.pt' \
--arch 'vgg_16_bn' \
--pruning_method 'L1'\
--dataset 'cifar10' \
--num_class 10 \
--batch_size 128

==> Preparing data..
Files already downloaded and verified
==> Building model..
VGG(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU(inplace=True)
    (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (norm3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu3): ReLU(inplace=True)
    (conv4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (norm4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu4): 

IDRiD

In [6]:
!python rank_generation.py \
--pretrain_dir './baseline/Baseline_resnet_56_IDRID/model_best.pth.tar' \
--arch 'resnet_56' \
--pruning_method 'Beta'\
--dataset 'IDRID' \
--num_class 3 \
--batch_size 32

==> Preparing data..
==> Building model..
ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layers): ModuleList()
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu2): ReLU(inplace=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_st

### Training and evaluating pruned model 

c10

In [5]:
!python tuning.py \
--job_dir './rank_conv/vgg_16_bn_L1_cifar10_limit1' \
--arch 'vgg_16_bn' \
--use_pretrain \
--pretrain_dir './baseline/Baseline_VGG_cifar10/vgg_16_bn.pt' \
--compress_rate [0.65]*7+[0.8]*5 \
--rank_conv_prefix './rank_conv/vgg_16_bn_L1_cifar10_limit1' \
--num_class 10\
--dataset 'cifar10'\
--epochs 5

02/22 01:40:15 PM | args = Namespace(arch='vgg_16_bn', batch_size=256, compress_rate='[0.65]*7+[0.8]*5', data_dir='./data/', dataset='cifar10', epochs=5, gpu='0', job_dir='./rank_conv/vgg_16_bn_L1_cifar10_limit1', learning_rate=0.01, lr_decay_step='50,100', momentum=0.9, num_class=10, pretrain_dir='./baseline/Baseline_VGG_cifar10/vgg_16_bn.pt', rank_conv_prefix='./rank_conv/vgg_16_bn_L1_cifar10_limit1', resume=False, test_model_dir='', test_only=False, use_pretrain=True, weight_decay=0.0005)
['*7']
['*5']
02/22 01:40:15 PM | compress_rate:[0.65, 0.65, 0.65, 0.65, 0.65, 0.65, 0.65, 0.8, 0.8, 0.8, 0.8, 0.8]
02/22 01:40:15 PM | ==> Building model..
02/22 01:40:20 PM | VGG(
  (features): Sequential(
    (conv0): Conv2d(3, 22, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (norm0): BatchNorm2d(22, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (conv1): Conv2d(22, 22, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (norm1): 

IDRID

In [7]:
!python tuning.py \
--job_dir './rank_conv/resnet_56_Beta_IDRID_limit1' \
--arch 'resnet_56' \
--use_pretrain \
--pretrain_dir './baseline/Baseline_resnet_56_IDRID/model_best.pth.tar' \
--compress_rate [0.]+[0.18]*29  \
--rank_conv_prefix './rank_conv/resnet_56_Beta_IDRID_limit1' \
--num_class 3\
--dataset 'IDRID'\
--epochs 5

02/22 01:44:00 PM | args = Namespace(arch='resnet_56', batch_size=256, compress_rate='[0.]+[0.18]*29', data_dir='./data/', dataset='IDRID', epochs=5, gpu='0', job_dir='./rank_conv/resnet_56_Beta_IDRID_limit1', learning_rate=0.01, lr_decay_step='50,100', momentum=0.9, num_class=3, pretrain_dir='./baseline/Baseline_resnet_56_IDRID/model_best.pth.tar', rank_conv_prefix='./rank_conv/resnet_56_Beta_IDRID_limit1', resume=False, test_model_dir='', test_only=False, use_pretrain=True, weight_decay=0.0005)
['*29']
02/22 01:44:00 PM | compress_rate:[0.0, 0.18, 0.18, 0.18, 0.18, 0.18, 0.18, 0.18, 0.18, 0.18, 0.18, 0.18, 0.18, 0.18, 0.18, 0.18, 0.18, 0.18, 0.18, 0.18, 0.18, 0.18, 0.18, 0.18, 0.18, 0.18, 0.18, 0.18, 0.18, 0.18]
02/22 01:44:00 PM | ==> Building model..
02/22 01:44:03 PM | ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
 

### Libraries version

In [8]:
!pip list

Package                       Version
----------------------------- ----------------------
absl-py                       1.4.0
aeppl                         0.0.33
aesara                        2.7.9
aiohttp                       3.8.4
aiosignal                     1.3.1
alabaster                     0.7.13
albumentations                1.2.1
altair                        4.2.2
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arviz                         0.12.1
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
async-timeout                 4.0.2
atari-py                      0.2.9
atomicwrites                  1.4.1
attrs                         22.2.0
audioread                     3.0.0
autograd                      1.5
Babel                         2.11.0
backcall                      0.2.0
beautifulsoup4                4.6.3
bleach                        6.0.0
blis